1\)

Importing Data

In [4]:
import numpy as np

from sklearn import datasets
iris = datasets.load_iris()
X = iris.data[:,0:4] ## from this only take features 0,1,2,3
Y = iris.target

print(X.shape)
print(Y)

(150, 4)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [16]:
def PGauss(mu, sig, x):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.) + 1e-300) )

def processing_for_NB(X, Y):
    U = np.zeros((X.shape[1], 3))
    S = np.zeros((X.shape[1], 3))
    PClass = []

    for i in range(3):
        PClass.append(np.array(np.where(Y==i)).shape[1]/Y.shape[0])
        for f in range(X.shape[1]):
            U[f,i] = X[np.where(Y==i),f].mean()
            S[f,i] = X[np.where(Y==i),f].std()

    return PClass, U, S
    
PClass, U, S = processing_for_NB(X,Y)
print(PClass)
print(U)
print(S)

def NB(X, PClass, U, S):
    Y = np.zeros((X.shape))
    for x, i in enumerate(X):
        



50
150
50
150
50
150
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
[[5.006 5.936 6.588]
 [3.428 2.77  2.974]
 [1.462 4.26  5.552]
 [0.246 1.326 2.026]]
[[0.34894699 0.51098337 0.62948868]
 [0.37525458 0.31064449 0.31925538]
 [0.17191859 0.46518813 0.54634787]
 [0.10432641 0.19576517 0.27188968]]
